In [1]:
import shutil
import pandas as pd
from pydantic import BaseModel, field_validator
from datetime import datetime
from sqlalchemy import create_engine
from typing import Optional
import json
from pathlib import Path

In [2]:
# ===========================================
# 1️⃣ Reader: Veriyi okuma işlemini kapsülleyen sınıf
# ===========================================
class DataReader:
    """Veri okuma işlemini soyutlayan base class"""
    def read(self) -> pd.DataFrame:
        raise NotImplementedError


class ParquetReader(DataReader):
    """Parquet dosyasını okuyabilen sınıf"""
    def __init__(self, file_path: str):
        self.file_path = file_path

    def read(self) -> pd.DataFrame:
        # Eğer dosya gzip ise direkt kopyalayıp parquet olarak açıyoruz
        if self.file_path.endswith(".gzip"):
            shutil.copy(self.file_path, self.file_path.replace(".gzip", ""))
            path = self.file_path.replace(".gzip", "")
        else:
            path = self.file_path
        return pd.read_parquet(path)


class ConfigReader:
    """Config json dosyasını okur"""
    def __init__(self, config_path: str):
        self.config_path = Path(config_path)

    def read(self) -> dict:
        text = self.config_path.read_text(encoding="utf-8")
        return json.loads(text)

In [3]:
# ===========================================
# 2️⃣ Pydantic Modelleri: Validation & Normalization
# ===========================================
class UserModel(BaseModel):
    user_id: float
    subscriber_id: Optional[float]
    country: Optional[str]
    has_email_contact_permission: Optional[bool]
    has_phone_contact_permission: Optional[bool]

    @field_validator("has_email_contact_permission", "has_phone_contact_permission", mode="before")
    def to_bool(cls, v):
        if v is None:
            return None
        if isinstance(v, str):
            return v.lower() in ["yes", "true", "1"]
        return bool(v)


class SessionModel(BaseModel):
    session_id: str
    user_id: Optional[float]
    user_agent: Optional[str]
    device_type: Optional[str]
    ip_address: Optional[str]
    utm_source: Optional[str]


class EventModel(BaseModel):
    request_id: str
    session_id: str
    funnel_id: str
    timestamp: datetime
    page_name: str
    search_query: Optional[str]
    destination_id: Optional[float]
    num_guests: Optional[float]


class HotelModel(BaseModel):
    hotel_id: int
    hotel_price: Optional[float]
    currency: Optional[str]

    @field_validator("hotel_price", mode="before")
    def clean_price(cls, v):
        if v is None:
            return None
        if isinstance(v, str):
            v = v.replace(",", ".").replace("$", "").strip()
        try:
            return float(v)
        except ValueError:
            return None


class PaymentModel(BaseModel):
    request_id: str
    payment_status: Optional[str]
    confirmation_number: Optional[str]

    @field_validator("payment_status", mode="before")
    def normalize_status(cls, v):
        if v is None:
            return None
        v = v.strip().lower()
        mapping = {
            "success": "completed",
            "done": "completed",
            "ok": "completed",
            "paid": "completed",
            "fail": "failed",
            "error": "failed"
        }
        return mapping.get(v, v)

    @field_validator("payment_status")
    def check_valid_values(cls, v):
        allowed = {"pending", "completed", "failed", "refunded", None}
        if v not in allowed:
            raise ValueError(f"Invalid payment status: {v}")
        return v

In [4]:
# ===========================================
# 3️⃣ Processor: Validation ve Normalizasyon işlemleri
# ===========================================
class FunnelProcessor:
    """Veriyi validate eder, normalize eder ve tabloya ayırır"""
    def validate_and_normalize(self, df: pd.DataFrame) -> dict:
        users, sessions, events, hotels, payments = [], [], [], [], []

        for row in df.to_dict(orient="records"):
            try:
                users.append(UserModel(**row).model_dump(exclude_none=True))
                sessions.append(SessionModel(**row).model_dump(exclude_none=True))
                events.append(EventModel(**row).model_dump(exclude_none=True))
                hotels.append(HotelModel(**row).model_dump(exclude_none=True))
                payments.append(PaymentModel(**row).model_dump(exclude_none=True))
            except Exception as e:
                print(f"Validation error: {e}")

        return {
            "users": pd.DataFrame(users).drop_duplicates(subset=['user_id']),
            "sessions": pd.DataFrame(sessions).drop_duplicates(subset=['session_id']),
            "events": pd.DataFrame(events),
            "hotels": pd.DataFrame(hotels).drop_duplicates(subset=['hotel_id']),
            "payments": pd.DataFrame(payments)
        }

In [5]:
# ===========================================
# 4️⃣ Writer: Veriyi MySQL’e yazma işlemi
# ===========================================
class MySQLWriter:
    """Normalize edilmiş tabloları MySQL’e yazar"""
    def __init__(self, config: dict):
        self.engine = create_engine(
            f"mysql+pymysql://{config['kullanici']}:{config['sifre']}@{config['host']}:{config['port']}/{config['veritabani']}"
        )

    def write(self, tables: dict):
        for name, df in tables.items():
            df.to_sql(name, con=self.engine, if_exists='replace', index=False)
            print(f"✅ {name} tablosu MySQL'e yazıldı.")

In [6]:
# ===========================================
# 5️⃣ Pipeline: Tüm adımları bir araya getiren sınıf
# ===========================================
class FunnelPipeline:
    """Reader → Processor → Writer akışını yöneten pipeline"""
    def __init__(self, data_path: str, config_path: str):
        self.reader = ParquetReader(data_path)
        self.config = ConfigReader(config_path).read()
        self.processor = FunnelProcessor()
        self.writer = MySQLWriter(self.config)

    def run(self):
        print("📥 Veriyi okuyoruz...")
        df_raw = self.reader.read()
        print("✅ Veri okundu. Validation ve normalize işlemi başlıyor...")
        normalized_tables = self.processor.validate_and_normalize(df_raw)
        print("✅ Normalize işlemi tamamlandı. MySQL'e yazılıyor...")
        self.writer.write(normalized_tables)
        print("🎉 Pipeline başarıyla tamamlandı!")

In [7]:
# ===========================================
# 6️⃣ Pipeline'ı çalıştır
# ===========================================
if __name__ == "__main__":
    pipeline = FunnelPipeline(
        data_path="//Users//sdedeoglu//Desktop//python//case_data.parquet.gzip",
        config_path="//Users//sdedeoglu//Desktop//python//config.json"
    )
    pipeline.run()

📥 Veriyi okuyoruz...
✅ Veri okundu. Validation ve normalize işlemi başlıyor...
✅ Normalize işlemi tamamlandı. MySQL'e yazılıyor...
✅ users tablosu MySQL'e yazıldı.
✅ sessions tablosu MySQL'e yazıldı.
✅ events tablosu MySQL'e yazıldı.
✅ hotels tablosu MySQL'e yazıldı.
✅ payments tablosu MySQL'e yazıldı.
🎉 Pipeline başarıyla tamamlandı!
